# Tratamento de Dados do Censo Escolar 

O Censo Escolar é o principal instrumento de coleta de informações da educação básica e a mais importante pesquisa estatística educacional brasileira. É coordenado pelo Inep e realizado em regime de colaboração entre as secretarias estaduais e municipais de educação e com a participação de todas as escolas públicas e privadas do país.

## Obtenção dos dados

Os dados podem ser obtidos através do seguinte link:

https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-escolar/resultados/2022

No link acima, deve-se selecionar os _Microdados da Educação Básica 2022_.

Para obter resultados de anos anteriores, basta alterar a última parte do link para o ano desejada.

**Importando as bibliotecas disponíveis**

Neste caso, utilizaremos apenas a biblioteca _pandas_.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

**Carregamento dos dados**

In [2]:
File2016 = 'microdados_ed_basica_2016.csv'
File2017 = 'microdados_ed_basica_2017.csv'
File2018 = 'microdados_ed_basica_2018.csv'
File2019 = 'microdados_ed_basica_2019.csv'
File2020 = 'microdados_ed_basica_2020.csv'
File2021 = 'microdados_ed_basica_2021.csv'
File2022 = 'microdados_ed_basica_2022.csv'

**Função para tratamento dos dados**

Na função abaixo, iremos realizar as seguintes operações:

1. _Unpivot_ dos dados, através da função `pd.melt`;
2. Carregamento dos dados, através da função `pd.read_csv`;
3. Filtro do município desejado e somente de escolas públicas - `.query('CO_MUNICIPIO==3550704 and TP_DEPENDENCIA==3')`
4. Resetar o index da _DataFrame_ após o filtro: `.reset_index()`;
5. Filtrar somente a partir da 14ª coluna do carregamento original (antes do _melt_): `iloc[:,:14]`
6. Remover colunas que não serão utilizadas na análise: `.drop(columns=['CO_ENTIDADE', 'TP_DEPENDENCIA'])`
7. Renomear colunas que surgiram (_variable, value_) através do nosso _melt_: `.rename(columns={'variable': 'DESCRIÇÃO', 'value': 'VALOR'})`
8. Por fim, o `return`do nosso _DataFrame_
 

In [3]:
def Clean_DataFrame(file):
    DataFrame = pd.melt(pd.read_csv(file, sep=';', encoding='latin1').query('CO_MUNICIPIO==3550704 and TP_DEPENDENCIA==3').reset_index().iloc[:, 14:].drop(
    columns=['CO_ENTIDADE', 'TP_DEPENDENCIA']), id_vars='NO_ENTIDADE').fillna(0).rename(
    columns={'variable': 'DESCRIÇÃO', 'value': 'VALOR'})
    return DataFrame

**Concatenando os DataFrames**

In [4]:
Ano = 2016
DataFrames = []
for File in [File2016, File2017, File2018, File2019, File2020, File2021, File2022]:  
    df = Clean_DataFrame(File)
    df['Ano'] = Ano
    DataFrames.append(df)
    Ano += 1
    
Final_DataFrame = pd.concat(DataFrames)

C:\Users\rafael.buzi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


**Obtendo os dados do Dicionário de Dados**

O dicionário de dados nos permitirá a análise somente de itens que tenham sido avaliados em 2022, e que sejam numéricos, conforme abaixo:

`Dicionario.query("Avaliação_2022=='s' and Tipo=='Num'", engine='python')`

Adicionalmente, o dicionário traz a descrição das siglas, bastante útil para a nossa avaliação.

In [5]:
File = 'dicionário_dados_educação_básica.xlsx'
Cols = 'Unnamed: 2','Unnamed: 3', 'Alteração de nomenclatura', 'Unnamed: 21'
Dicionario = pd.read_excel(File, usecols=Cols)
Dicionario.columns = ['Sigla', 'Descrição', 'Tipo', 'Avaliação_2022']
Dicionario = Dicionario.query("Avaliação_2022=='s' and Tipo=='Num'", 
                              engine='python').drop('Avaliação_2022', axis=1).reset_index(drop=True)

**Merge**

Aqui, faremos um _inner join_ entre os dados dos censos de 2016 a 2022 `Final_DataFrame`e o nosso `Dicionario`.

In [6]:
MicroDados = pd.merge(Final_DataFrame, Dicionario, left_on='DESCRIÇÃO', right_on='Sigla', how='inner').fillna(0).drop(columns=['DESCRIÇÃO', 'Tipo'])

MicroDados['ValorAjustado'] = pd.to_numeric(MicroDados['VALOR'], errors='coerce')

Uma vez que criamos uma coluna chamada `ValorAjustado`, com o parâmetro de `errors=coerce`, os valores que não foram convertidos devidamente em numéricos permancerão vazios. Desta forma, com o código abaixo, poderemos obter as siglas que não devem permanecer na nossa análise:

In [7]:
MicroDados[MicroDados['ValorAjustado'].isnull()]['Sigla'].value_counts()

NU_TELEFONE    28
NU_DDD         28
Name: Sigla, dtype: int64

In [13]:
MicroDados[MicroDados['ValorAjustado'].isnull()]['VALOR'].value_counts()

.    56
Name: VALOR, dtype: int64

Nos 3 blocos de código acima, podemos ver que os problemas se dão no preenchimento das siglas `NU_TELEFONE` e `NU_DDD`, que possuem `.`como valor.

No bloco de código abaixo, iremos remover tais siglas, uma vez que são irrelevantes na nossa análise>

In [14]:
MicroDados = MicroDados.query("Sigla!='NU_TELEFONE' and Sigla!='NU_DDD'", 
                              engine='python').drop('VALOR', axis=1)

Por fim, iremos agrupar os dados, a fim de termos o resultado do município como um todo

In [19]:
Group = MicroDados.groupby(['Ano', 'Sigla', 'Descrição'])['ValorAjustado'].sum().reset_index()
Group.Ano = pd.to_datetime(Group.Ano, format='%Y')

Visão final dos dados

In [23]:
Group.head(5)

,Ano,Sigla,Descrição,ValorAjustado
0,2016-01-01,CO_ESCOLA_SEDE_VINCULADA,Código da escola sede,0.0
1,2016-01-01,CO_IES_OFERTANTE,Código da IES vinculada à escola,0.0
2,2016-01-01,CO_LINGUA_INDIGENA_1,Educação Indígena - Língua em que o ensino é m...,0.0
3,2016-01-01,CO_LINGUA_INDIGENA_2,Educação Indígena - Língua em que o ensino é m...,0.0
4,2016-01-01,CO_LINGUA_INDIGENA_3,Educação Indígena - Língua em que o ensino é m...,0.0
